In [189]:
import os
import pandas as pd
import keras

from sklearn.preprocessing import MinMaxScaler

Latitude	Longitude	Pressure	Wind Speed

In [190]:
def get_data_frame():
    current_folder = os.getcwd()
    file_path = os.path.join(current_folder,'..' ,'resource', 'RSMC_Best_Track_Data.csv')
    df = pd.read_csv(file_path)

    # ---------------------------

    direction_30_mapping = {'(symmetric circle)': 1, 'Northeast (NE)': 6, 'South (S)': 3,
                            'East (E)': 5, 'Southeast (SE)': 9, 'West (W)': 4, 'North (N)': 2,
                            'Northwest (NW)': 6, 'Southwest (SW)': 8}
    df['Direction of the longest radius of 30kt winds or greater'] = df[
        'Direction of the longest radius of 30kt winds or greater'].map(direction_30_mapping)

    direction_50_mapping = {'(symmetric circle)': 1, 'Northeast (NE)': 6, 'South (S)': 3,
                            'East (E)': 5, 'Southeast (SE)': 9, 'West (W)': 4, 'North (N)': 2,
                            'Northwest (NW)': 6, 'Southwest (SW)': 8,
                            'No direction (Longest radius of 50kt winds is 0)': 9}
    df['Direction of the longest radius of 50kt winds or greater'] = df[
        'Direction of the longest radius of 50kt winds or greater'].map(direction_50_mapping)

    indicator_mapping = {'#': 1, ' ': 0}
    df['Indicator of landfall or passage'] = df['Indicator of landfall or passage'].map(indicator_mapping)

    # ----------------------
    df_id_name = df[['International number ID', 'Name of the storm']]
    df = df.drop(['International number ID', 'Name of the storm', 'Grade'], axis=1)
    # df = df.drop(['Grade'], axis=1)
    
    # -----------------------

    one_hot_encoded = pd.get_dummies(df['Direction of the longest radius of 50kt winds or greater'],
                                     prefix='Direction of the longest radius of 50kt winds or greater', dtype=int)
    df = pd.concat([df, one_hot_encoded], axis=1)

    one_hot_encoded2 = pd.get_dummies(df['Direction of the longest radius of 30kt winds or greater'],
                                      prefix='Direction of the longest radius of 30kt winds or greater', dtype=int)
    df = pd.concat([df, one_hot_encoded2], axis=1)
    df = df.drop(['Direction of the longest radius of 50kt winds or greater',
                  'Direction of the longest radius of 30kt winds or greater'], axis=1)

    # ----------------------

    df['Time of analysis'] = pd.to_datetime(df['Time of analysis'])
    df['Time of analysis'] = df['Time of analysis'].apply(lambda x: x.timestamp())

    df_without_id_name = df# .drop(['Unnamed: 0'], axis=1)
    
    
    
    print('max latitude: ', df['Latitude of the center'].max())
    print('min latitude: ', df['Latitude of the center'].min())
    print('max longitude: ', df['Longitude of the center'].max())
    print('min longitude: ', df['Longitude of the center'].min())
    
    print('max pressure: ', df['Central pressure'].max())
    print('min pressure: ', df['Central pressure'].min())
    print('max wind speed: ', df['Maximum sustained wind speed'].max())
    print('min wind speed: ', df['Maximum sustained wind speed'].min())
    # -----------------------
    return df_without_id_name, df_id_name

Latitude of the center	Longitude of the center

In [191]:
def min_max_scaler(df: pd.DataFrame):
    
    columns_to_normalize = df.columns[:]

    # 实例化 MinMaxScaler
    scaler = MinMaxScaler()

    # 对选择的列进行归一化
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

    scale_min = scaler.data_min_
    scale_max = scaler.data_max_

    return df, scale_min, scale_max

In [192]:
def get_X_y(df: pd.DataFrame):
    y = df[["Latitude of the center", "Longitude of the center"]]
    X = df.drop(["Latitude of the center", "Longitude of the center"], axis=1)
    return X, y

In [193]:
df_without_id_name, df_id_name = get_data_frame()

max latitude:  69.0
min latitude:  1.4
max longitude:  188.0
min longitude:  95.0
max pressure:  1022
min pressure:  870
max wind speed:  140.0
min wind speed:  0.0


In [166]:
df_without_id_name, scale_min, scale_max = min_max_scaler(df_without_id_name)
X, y = get_X_y(df_without_id_name)

In [167]:
df_id_name.sample(10)

,International number ID,Name of the storm
59761,1312,TRAMI
46190,9713,WINNIE
16034,6723,PATSY
47299,9815,FAITH
16693,6802,KIM
25008,7618,GEORGIA
48967,104,UTOR
22567,7402,AMY
35567,8720,LYNN
24729,7609,THERESE


In [168]:
df_id_name['International number ID'] = df_id_name['International number ID'].astype(str)
df_id_name['International number ID'] = df_id_name['International number ID'].apply(lambda x: x.zfill(4))
df_id_name['Name of the storm'] = df_id_name['Name of the storm'].astype(str)

In [169]:
df_id_name.sample(10)

,International number ID,Name of the storm
26514,7813,nan
66577,2010,HAISHEN
46541,9721,FRITZ
1665,5301,IRMA
66927,2020,ATSANI
10389,6228,KAREN
63631,1713,HATO
28896,8024,ED
6492,5901,RUBY
19154,7020,KATE


In [170]:
df_max_min_id_name = pd.concat([df_id_name, df_without_id_name], axis=1)

In [171]:
df_max_min_id_name

,International number ID,Name of the storm,Time of analysis,Latitude of the center,Longitude of the center,Central pressure,Maximum sustained wind speed,The longest radius of 50kt winds or greater,The shortest radius of 50kt winds or greater,The longest radius of 30kt winds or greater,...,Direction of the longest radius of 50kt winds or greater_8.0,Direction of the longest radius of 50kt winds or greater_9.0,Direction of the longest radius of 30kt winds or greater_1.0,Direction of the longest radius of 30kt winds or greater_2.0,Direction of the longest radius of 30kt winds or greater_3.0,Direction of the longest radius of 30kt winds or greater_4.0,Direction of the longest radius of 30kt winds or greater_5.0,Direction of the longest radius of 30kt winds or greater_6.0,Direction of the longest radius of 30kt winds or greater_8.0,Direction of the longest radius of 30kt winds or greater_9.0
0,5101,nan,0.000000,0.275148,0.467742,0.921053,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5101,nan,0.000009,0.275148,0.467742,0.921053,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5101,nan,0.000019,0.319527,0.506452,0.855263,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5101,nan,0.000028,0.349112,0.548387,0.815789,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5101,nan,0.000038,0.387574,0.597849,0.815789,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68745,2303,GUCHOL,0.999962,0.560651,0.854839,0.789474,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68746,2303,GUCHOL,0.999972,0.559172,0.876344,0.763158,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68747,2303,GUCHOL,0.999981,0.565089,0.893548,0.750000,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68748,2303,GUCHOL,0.999991,0.565089,0.909677,0.750000,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [172]:

no_na = df_max_min_id_name.dropna()

no_na

,International number ID,Name of the storm,Time of analysis,Latitude of the center,Longitude of the center,Central pressure,Maximum sustained wind speed,The longest radius of 50kt winds or greater,The shortest radius of 50kt winds or greater,The longest radius of 30kt winds or greater,...,Direction of the longest radius of 50kt winds or greater_8.0,Direction of the longest radius of 50kt winds or greater_9.0,Direction of the longest radius of 30kt winds or greater_1.0,Direction of the longest radius of 30kt winds or greater_2.0,Direction of the longest radius of 30kt winds or greater_3.0,Direction of the longest radius of 30kt winds or greater_4.0,Direction of the longest radius of 30kt winds or greater_5.0,Direction of the longest radius of 30kt winds or greater_6.0,Direction of the longest radius of 30kt winds or greater_8.0,Direction of the longest radius of 30kt winds or greater_9.0
25272,7701,PATSY,0.360915,0.062130,0.686022,0.789474,0.357143,0.000000,0.00,0.126506,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25273,7701,PATSY,0.360924,0.069527,0.679570,0.828947,0.357143,0.000000,0.00,0.126506,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25274,7701,PATSY,0.360934,0.076923,0.672043,0.855263,0.357143,0.000000,0.00,0.126506,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25275,7701,PATSY,0.360943,0.082840,0.664516,0.868421,0.321429,0.000000,0.00,0.096386,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25276,7701,PATSY,0.360953,0.087278,0.655914,0.868421,0.321429,0.000000,0.00,0.096386,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68727,2303,GUCHOL,0.999792,0.337278,0.404301,0.690789,0.428571,0.246154,0.24,0.301205,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
68728,2303,GUCHOL,0.999801,0.357988,0.417204,0.723684,0.392857,0.246154,0.24,0.337349,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
68729,2303,GUCHOL,0.999811,0.369822,0.433333,0.723684,0.392857,0.215385,0.20,0.337349,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
68730,2303,GUCHOL,0.999820,0.392012,0.450538,0.723684,0.392857,0.184615,0.16,0.337349,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


max latitude:  69.0  
min latitude:  1.4  
  
max longitude:  188.0  
min longitude:  95.0  
  
max pressure:  1022  
min pressure:  870  
  
max wind speed:  140.0  
min wind speed:  0.0  

In [141]:
def min_max_denormalize(normalized_data, min_value, max_value):
    denormalized_data = normalized_data * (max_value - min_value) + min_value
    return denormalized_data

# 使用保存的最小/最大值进行去归一化
# 假设 normalized_latitude 和 normalized_longitude 是需要去归一化的值
# scale_max[2]
denormalized_latitude = min_max_denormalize(0.369822, 1.4, 69.0)
denormalized_longitude = min_max_denormalize(0.433333, 95.0, 188.0)

print(denormalized_latitude, denormalized_longitude)

26.399967199999995 135.299969


In [173]:
no_na.to_csv(r'C:\CODE\tropical_cyclone_prediction\resource\min_max_with_id_name.csv')

In [175]:
for_model = no_na.drop(['International number ID', 'Name of the storm', 'Latitude of the center', 'Longitude of the center'], axis=1)

In [181]:
for_model = for_model.tail(5)
for_model

,Time of analysis,Central pressure,Maximum sustained wind speed,The longest radius of 50kt winds or greater,The shortest radius of 50kt winds or greater,The longest radius of 30kt winds or greater,The shortest radius of 30kt winds or greater,Indicator of landfall or passage,Direction of the longest radius of 50kt winds or greater_1.0,Direction of the longest radius of 50kt winds or greater_2.0,...,Direction of the longest radius of 50kt winds or greater_8.0,Direction of the longest radius of 50kt winds or greater_9.0,Direction of the longest radius of 30kt winds or greater_1.0,Direction of the longest radius of 30kt winds or greater_2.0,Direction of the longest radius of 30kt winds or greater_3.0,Direction of the longest radius of 30kt winds or greater_4.0,Direction of the longest radius of 30kt winds or greater_5.0,Direction of the longest radius of 30kt winds or greater_6.0,Direction of the longest radius of 30kt winds or greater_8.0,Direction of the longest radius of 30kt winds or greater_9.0
68727,0.999792,0.690789,0.428571,0.246154,0.24,0.301205,0.25,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
68728,0.999801,0.723684,0.392857,0.246154,0.24,0.337349,0.25,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
68729,0.999811,0.723684,0.392857,0.215385,0.20,0.337349,0.25,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
68730,0.999820,0.723684,0.392857,0.184615,0.16,0.337349,0.25,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
68731,0.999830,0.756579,0.357143,0.000000,0.00,0.337349,0.25,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [177]:
model = keras.models.load_model(r'C:\CODE\tropical_cyclone_prediction\model\CNN-LSTM-20-200.keras')

In [182]:
pred = model.predict(for_model)
pred

1/1 [==============================] - 0s 34ms/step


array([[[0.7805075 ],
        [0.5950083 ],
        [0.48118854]],

       [[0.77994746],
        [0.6167106 ],
        [0.48704   ]],

       [[0.77641976],
        [0.6257947 ],
        [0.49996945]],

       [[0.77195615],
        [0.6317512 ],
        [0.512446  ]],

       [[0.7526086 ],
        [0.62905663],
        [0.5555898 ]]], dtype=float32)

In [187]:
result = pred[:, 1:, :]
result = [i.flatten().tolist() for i in result]

In [188]:
for i in result:
    print(i)

[0.5950083136558533, 0.4811885356903076]
[0.6167106032371521, 0.4870400130748749]
[0.6257947087287903, 0.4999694526195526]
[0.6317511796951294, 0.5124459862709045]
[0.6290566325187683, 0.5555897951126099]
